In [1]:
import deepface
deepface.__version__

'0.0.96'

In [3]:
from deepface import DeepFace
import os
import glob
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm

25-12-10 09:20:40 - Directory /root/.deepface has been created
25-12-10 09:20:40 - Directory /root/.deepface/weights has been created


In [4]:
DATASET = "/content/dataset_augmented"

classes = os.listdir(DATASET)
print("Detected classes:", classes)

Detected classes: ['Brian Alexander', 'Melvern', 'Jason']


In [5]:
embeddings = []
labels = []

for person in classes:
    folder = os.path.join(DATASET, person)
    image_files = glob.glob(folder + "/*.jpg")

    print(f"Processing {person} : {len(image_files)} images")

    for img_path in tqdm(image_files):

        try:
            rep = DeepFace.represent(
                img_path=img_path,
                model_name="ArcFace",
                enforce_detection=False
            )

            # DeepFace returns a list → get first dict → extract embedding vector
            emb = np.array(rep[0]["embedding"])

            embeddings.append(emb)
            labels.append(person)

        except Exception as e:
            print("Skipped:", img_path, "| Reason:", e)

Processing Brian Alexander : 10 images


  0%|          | 0/10 [00:00<?, ?it/s]

25-12-10 09:20:46 - 🔗 arcface_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5 to /root/.deepface/weights/arcface_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5
To: /root/.deepface/weights/arcface_weights.h5

  0%|          | 0.00/137M [00:00<?, ?B/s]
 14%|█▍        | 19.4M/137M [00:00<00:00, 193MB/s]
 29%|██▊       | 39.3M/137M [00:00<00:00, 194MB/s]
 43%|████▎     | 59.2M/137M [00:00<00:00, 164MB/s]
 56%|█████▌    | 76.5M/137M [00:00<00:00, 143MB/s]
 69%|██████▉   | 94.4M/137M [00:00<00:00, 153MB/s]
 82%|████████▏ | 113M/137M [00:00<00:00, 161MB/s] 
100%|██████████| 137M/137M [00:00<00:00, 139MB/s]
100%|██████████| 10/10 [00:33<00:00,  3.36s/it]


Processing Melvern : 44 images


100%|██████████| 44/44 [01:09<00:00,  1.57s/it]


Processing Jason : 10 images


100%|██████████| 10/10 [03:00<00:00, 18.01s/it]


In [6]:
X = np.array(embeddings)
y = np.array(labels)

print("Embedding matrix:", X.shape)
print("Labels:", y.shape)

Embedding matrix: (64, 512)
Labels: (64,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [8]:
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [9]:
y_pred = svm.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy:", acc)

Test Accuracy: 0.9230769230769231


In [10]:
import joblib

joblib.dump(svm, "face_classifier_svm.pkl")
np.save("embeddings_labels.npy", y)

In [11]:
def predict_face(img_path):

    rep = DeepFace.represent(
        img_path,
        model_name="ArcFace",
        enforce_detection=False
    )

    if isinstance(rep, list):
        emb = np.array(rep[0]["embedding"])
    elif isinstance(rep, dict):
        emb = np.array(rep["embedding"])
    else:
        raise ValueError("Unknown embedding format:", rep)

    emb = emb.reshape(1, -1)

    pred = svm.predict(emb)[0]
    prob = svm.predict_proba(emb)[0].max()

    return pred, prob

In [12]:
image_to_predict = "/content/WhatsApp Image 2025-12-10 at 16.28.02_cb4337b5.jpg"
predicted_class, confidence_score = predict_face(image_to_predict)
print(f"Predicted class: {predicted_class}, Confidence: {confidence_score:.4f}")

Predicted class: Brian Alexander, Confidence: 0.6651
